In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import walk
from os.path import splitext

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD

import custom_loss_functions as cust_loss

In [ ]:
def simple_model():
    """Define simple model"""
    # create model
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(96,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # show info
    model.summary()
    # compile model
    model.compile(loss = cust_loss.mse_wrap_angle,
                  optimizer = 'adam',
                  metrics = [cust_loss.mae_wrap_angle])
    return model

## Training

In [ ]:
# define training parameters
batch_size = 32
num_epochs = 30

In [ ]:
# define dataset directory
data_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/hdf5/' 
# load filelist
filelist = []
for (dirpath, dirnames, filenames) in walk(data_dir):
    filelist.extend(filenames)
_, file_ext = splitext(filelist[0])
    
# load column labels only
if file_ext == '.h5':
    column_label = pd.read_hdf(data_dir+filelist[0], nrows=1).columns.tolist()
else:
    column_label = pd.read_csv(data_dir+filelist[0], nrows=1).columns.tolist()

feature_label = [s for s in column_label if ("ILD"  in s or "ITD" in s or "IC" in s)]
target_label = 'Localization_Azimuth'

In [ ]:
# normalization + train_test_split

# load data
data = pd.read_hdf(data_dir+filelist[62])
#test_data = pd.read_csv(data_dir+filelist[61])

# define scaler
minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
minmax_scaler.partial_fit(data[feature_label])
#minmax_scaler.partial_fit(test_data[feature_label])

#std_scale = preprocessing.StandardScaler().fit(data[feature_label])

# perform train_test_split
train, test = train_test_split(data, test_size=0.2)

# perform scaling
train_minmax = minmax_scaler.transform(train[feature_label])
test_minmax = minmax_scaler.transform(test[feature_label])
#train_std = std_scale.transform(train[feature_label])
#test_std = std_scale.transform(test[feature_label])

x_train = train_minmax
y_train = train[target_label]
x_test = test_minmax
y_test = test[target_label]

In [ ]:
# model.fit on entire dataset

model = simple_model()

history = model.fit(x_train, y_train,
                    #batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    validation_split=0.2)

## Evaluation

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test mae w wrap:', score[1])


# plot train history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
#test_data = pd.read_csv(data_dir+filelist[68])
test_features = test_data[feature_label].values
test_features_minmax = minmax_scaler.transform(test_features)
test_targets = test_data[target_label].values


test_results = model.predict(test_features_minmax)

In [ ]:
test_results = model.predict(test_minmax)

In [ ]:
test_results[0:100]

In [ ]:
# Save history to json file
import json
with open('file.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
from keras.models import load_model

model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [ ]:
# evaluate model
score = model.evaluate(test_features, test_targets, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test accuracy:', score[2])